## Overview
The schema for this dataset is shown. As discussed in the steps for data preparation there is a common pattern to preparing operational business data for analysis. There is always a timestamp that captures when the data was captured. Because the objective of this analysis is to segment revenue by various factors, we will define these factors and the attributes for each factor. The factors are customers, products and geography. The sequence of processing steps in this notebook follows the description of the key steps in the concepts document.

## Schema for the Olist Dataset

![](../images/olist_schema.png)

In [ ]:
import pandas as pd
fp_orders = "../data/olist_raw/olist_orders_dataset.csv"
fp_order_items = "../data/olist_raw/olist_order_items_dataset.csv"
df_orders = pd.read_csv(fp_orders)
df_order_items = pd.read_csv(fp_order_items)


## Filter and Aggregate Steps for Customer Data

In [ ]:
cols = df_orders.columns.tolist() + df_order_items.columns.tolist()

In [ ]:
cols

In [ ]:
cols_needed = ["order_id",  'customer_id', "order_purchase_timestamp", "order_item_id", "product_id", "price"]

In [ ]:
df_daily_ord_raw = pd.merge(df_orders, df_order_items, on = "order_id")

In [ ]:
df_daily_ord_raw = df_daily_ord_raw[cols_needed]

In [ ]:
df_daily_ord_raw["order_purchase_timestamp"] = pd.to_datetime(df_daily_ord_raw["order_purchase_timestamp"])

In [ ]:
df_daily_ord_raw

In [ ]:
fp_customer = "../data/olist_raw/olist_customers_dataset.csv"
df_cust = pd.read_csv(fp_customer)
cust_cols = ["customer_id","customer_zip_code_prefix", "customer_city", "customer_state"]
df_cust = df_cust[cust_cols]

In [ ]:
df_daily_ord_raw =  pd.merge(df_daily_ord_raw, df_cust, on = "customer_id")

In [ ]:
df_daily_ord_raw = df_daily_ord_raw.sort_values(by="order_id").reset_index()
fp_daily_ord_raw = "../data/olist_prepared/olist_daily_orders_prepared.csv"
df_daily_ord_raw.to_csv(fp_daily_ord_raw, index=False)

In [ ]:
ord_qty_vc = df_daily_ord_raw.order_id.value_counts().values

In [ ]:
ord_qty_vc

In [ ]:
df_gb = df_daily_ord_raw.groupby([df_daily_ord_raw.order_purchase_timestamp.dt.year,\
                                  df_daily_ord_raw.order_purchase_timestamp.dt.day_of_year])

In [ ]:
df_prod_purchases = df_daily_ord_raw.product_id.value_counts().reset_index()

In [ ]:
len(df_daily_ord_raw.product_id.unique())

In [ ]:
from matplotlib import pyplot as plt 

In [ ]:
df_prod_purchases["count"].quantile(q=0.8)

## Filter and Aggregate Steps for Product Data

In [ ]:
df_prod_purchases = df_prod_purchases.sort_values(by="count", ascending=False)
fp_prod_purch = "../data/olist_prepared/olist_2_year_prod_purchases.csv"
df_prod_purchases["rank"] = df_prod_purchases.index + 1
df_prod_purchases.to_csv(fp_prod_purch, index=False)

In [ ]:
df_prod_purchases.head(20)

In [ ]:
df_prod_purchases["count"].unique().shape

In [ ]:
df_cust_order_count = df_daily_ord_raw.groupby(["customer_id"])["order_id"].count().reset_index()

In [ ]:
df_cust_order_count.columns = ["customer_id", "order_count"]

In [ ]:
df_cust_order_count = df_cust_order_count.sort_values(by="order_count", ascending=False).reset_index()